## Final Project Submission
* Student name: Charles Owiti, Noel Seda, Michelle Chekwoti , Alex Mwera
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Mildred Jepkosgei

## Business Problem: 

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

Objectives:
- Analyse movie ratings, number of subscriptions in order to determine the best movies.
- Business recommendations for the new studio for popular genres that will attract large audiences
- Analyse Feedback of users.
- Analysis of high rates genres that could be successful.

The Aim: is to analyse the movie industry to find the gaps in original video content that will allow our new studio to create succesful films.

## Data Understanding

In [4]:
# Importing the relevant libraries
import pandas as pd
import sqlite3

In [5]:
# Connecting to the IMDB database
conn = sqlite3.connect('im.db')
cur = conn.cursor()

In [6]:
# View database tables
tables_query = """SELECT name 
                      AS 'Table Names' 
                      FROM sqlite_master 
                      WHERE type='table';"""

pd.read_sql(tables_query, conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


They are 7 tables in the database and we will focus on the Movie_basics and Movie_ratings.

In [9]:
movie_basics_query = """SELECT * FROM movie_basics ;"""

pd.read_sql(movie_basics_query, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [10]:
movie_ratings_query = """SELECT * FROM movie_ratings ;"""

pd.read_sql(movie_ratings_query, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


The Movie basics table consists of:
- `movie_Id`: The unique id for each movie and the Primary Key.
- `primary_title`: The title of the movie.
- `genres`: The genre(s) of the movie.
  
The Movie ratings table consists of:
- `average_rating`: The average rating given by users.
- `num_votes`: The number of votes a movie has received.
    
Our goal is to analyze the relationship between ratings, votes, and genres.

## Data Understanding

## Data Visualization